# Машинное обучение - Лабораторная работа 5

**Выполнила:**  
Идрисова Лена  


### Импорты и первичная настройка

In [ ]:
import numpy as np
import pandas as pd
import gdown
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns


## Advertising

### 1. Загрузка данных


In [ ]:
url = '   '
output = 'advertising_clean.csv'
gdown.download(url, output, quiet=False)
Advertising = pd.read_csv('/content/advertising_clean.csv')
Advertising.head()

Downloading...
From: https://drive.google.com/uc?export=download&id=1Y7MM-B-QaSWuzgQxjt6iPvezZC3Zzjs8
To: /content/advertising_clean.csv
100%|██████████| 14.8k/14.8k [00:00<00:00, 18.8MB/s]


,TV,radio,newspaper,sales
0,0.969852,0.981522,1.778945,1.552053
1,-1.197376,1.082808,0.669579,-0.696046
2,0.052050,1.217855,1.286405,0.860330
3,0.394182,-0.841614,1.281802,-0.215683
4,-1.045577,0.643905,-0.324708,-0.427043


### 2. Разбиваем датасет на train/test
Можно выделить валидационную выборку, но для упрощённого случая
будем использовать train_test_split и финальный тест для сравнения моделей.

In [ ]:
X = Advertising.drop('sales', axis=1)
y = Advertising['sales']

# Разделение в соотношении 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
)

print("Размер обучающей выборки:", X_train.shape, y_train.shape)
print("Размер тестовой выборки:", X_test.shape, y_test.shape)


Размер обучающей выборки: (150, 3) (150,)
Размер тестовой выборки: (38, 3) (38,)


### 3. Модель: Случайный лес (RandomForestClassifier)

In [ ]:
# Импорт регрессионных моделей (уже импортированы ранее)

# Параметры для RandomForestRegressor
rf_params = [
    {'n_estimators': 100, 'max_depth': None, 'max_features': 'sqrt'},
    {'n_estimators': 200, 'max_depth': 10,   'max_features': 'sqrt'},
    {'n_estimators': 50,  'max_depth': 5,    'max_features': 'log2'},
]

rf_models = {}
for i, params in enumerate(rf_params, start=1):
    rf = RandomForestRegressor(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        max_features=params['max_features'],
    )
    rf.fit(X_train, y_train)
    rf_models[f"RF_model_{i}"] = rf
    print(f"Обучена RF_model_{i} с параметрами {params}")

Обучена RF_model_1 с параметрами {'n_estimators': 100, 'max_depth': None, 'max_features': 'sqrt'}
Обучена RF_model_2 с параметрами {'n_estimators': 200, 'max_depth': 10, 'max_features': 'sqrt'}
Обучена RF_model_3 с параметрами {'n_estimators': 50, 'max_depth': 5, 'max_features': 'log2'}


### 4. Модель: Градиентный бустинг (GradientBoostingClassifier)

In [ ]:
# Параметры для GradientBoostingRegressor
gb_params = [
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 4},
    {'n_estimators': 50,  'learning_rate': 0.2,  'max_depth': 2},
]

gb_models = {}
for i, params in enumerate(gb_params, start=1):
    gb = GradientBoostingRegressor(
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
    )
    gb.fit(X_train, y_train)
    gb_models[f"GB_model_{i}"] = gb
    print(f"Обучена GB_model_{i} с параметрами {params}")

Обучена GB_model_1 с параметрами {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3}
Обучена GB_model_2 с параметрами {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 4}
Обучена GB_model_3 с параметрами {'n_estimators': 50, 'learning_rate': 0.2, 'max_depth': 2}


### 5. Сравнение качества на тестовой выборке

In [ ]:
def evaluate_regressor(model, X_test, y_test, model_name=""):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"===== {model_name} =====")
    print(f"MSE: {mse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R² Score: {r2:.4f}")
    print("-"*40)

# Оценим все модели RandomForestRegressor
for name, model in rf_models.items():
    evaluate_regressor(model, X_test, y_test, name)

# Оценим все модели GradientBoostingRegressor
for name, model in gb_models.items():
    evaluate_regressor(model, X_test, y_test, name)

===== RF_model_1 =====
MSE: 0.0338
MAE: 0.1463
R² Score: 0.9646
----------------------------------------
===== RF_model_2 =====
MSE: 0.0365
MAE: 0.1578
R² Score: 0.9617
----------------------------------------
===== RF_model_3 =====
MSE: 0.0837
MAE: 0.2375
R² Score: 0.9123
----------------------------------------
===== GB_model_1 =====
MSE: 0.0149
MAE: 0.0947
R² Score: 0.9844
----------------------------------------
===== GB_model_2 =====
MSE: 0.0191
MAE: 0.1099
R² Score: 0.9800
----------------------------------------
===== GB_model_3 =====
MSE: 0.0178
MAE: 0.1024
R² Score: 0.9813
----------------------------------------


### 6. Итоги и сравнительная оценка

GradientBoostingRegressor является более эффективным алгоритмом для данной задачи регрессии по сравнению с RandomForestRegressor, демонстрируя более низкие значения MSE и MAE, а также более высокие значения R² Score.

Оптимальная конфигурация для RandomForestRegressor (RF_model_2) включает увеличение числа деревьев до 200, ограничение глубины до 10 и использование 'sqrt' для параметра max_features.

Для GradientBoostingRegressor наилучшие результаты были достигнуты при использовании 100 деревьев, learning_rate 0.1 и max_depth 3, что обеспечивает баланс между скоростью обучения и точностью модели.



## Heart


### 1. Загрузка данных


In [ ]:
url = 'https://drive.google.com/uc?export=download&id=1bZinjn1gh3ZovYM2e6m_lwdkScYsooDG'
output = 'heart_new.csv'
gdown.download(url, output, quiet=False)
Heart = pd.read_csv('/content/heart_new.csv')
Heart.head()

Downloading...
From: https://drive.google.com/uc?export=download&id=1bZinjn1gh3ZovYM2e6m_lwdkScYsooDG
To: /content/heart_new.csv
100%|██████████| 7.55k/7.55k [00:00<00:00, 15.6MB/s]


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
1,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
2,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
3,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
4,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1


### 2. Разбиваем датасет на train/test
Можно выделить валидационную выборку, но для упрощённого случая
будем использовать train_test_split и финальный тест для сравнения моделей.

In [ ]:
X = Heart.drop('target', axis=1)
y = Heart['target']

# Разделение в соотношении 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
)

print("Размер обучающей выборки:", X_train.shape, y_train.shape)
print("Размер тестовой выборки:", X_test.shape, y_test.shape)


Размер обучающей выборки: (161, 13) (161,)
Размер тестовой выборки: (41, 13) (41,)


### 3. Модель: Случайный лес (RandomForestClassifier)

In [ ]:
# Перебираем несколько вариантов гиперпараметров
# (В реальности можно организовать GridSearchCV или RandomizedSearchCV для перебора параметров.)

rf_params = [
    {'n_estimators': 100, 'max_depth': None, 'max_features': 'sqrt'},
    {'n_estimators': 200, 'max_depth': 10,   'max_features': 'sqrt'},
    {'n_estimators': 50,  'max_depth': 5,    'max_features': 'log2'},
]

rf_models = {}
for i, params in enumerate(rf_params, start=1):
    rf = RandomForestClassifier(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        max_features=params['max_features'],
    )
    rf.fit(X_train, y_train)
    rf_models[f"RF_model_{i}"] = rf
    print(f"Обучена RF_model_{i} с параметрами {params}")

Обучена RF_model_1 с параметрами {'n_estimators': 100, 'max_depth': None, 'max_features': 'sqrt'}
Обучена RF_model_2 с параметрами {'n_estimators': 200, 'max_depth': 10, 'max_features': 'sqrt'}
Обучена RF_model_3 с параметрами {'n_estimators': 50, 'max_depth': 5, 'max_features': 'log2'}


### 4. Модель: Градиентный бустинг (GradientBoostingClassifier)

In [ ]:
# Несколько вариантов гиперпараметров для GBM
gb_params = [
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 4},
    {'n_estimators': 50,  'learning_rate': 0.2,  'max_depth': 2},
]

gb_models = {}
for i, params in enumerate(gb_params, start=1):
    gb = GradientBoostingClassifier(
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
    )
    gb.fit(X_train, y_train)
    gb_models[f"GB_model_{i}"] = gb
    print(f"Обучена GB_model_{i} с параметрами {params}")

Обучена GB_model_1 с параметрами {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3}
Обучена GB_model_2 с параметрами {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 4}
Обучена GB_model_3 с параметрами {'n_estimators': 50, 'learning_rate': 0.2, 'max_depth': 2}


### 5. Сравнение качества на тестовой выборке

In [ ]:
def evaluate_model(model, X_test, y_test, model_name=""):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"===== {model_name} =====")
    print(f"Accuracy: {acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print("-"*40)

# Оценим все модели RandomForestClassifier
for name, model in rf_models.items():
    evaluate_model(model, X_test, y_test, name)

# Оценим все модели GradientBoostingClassifier
for name, model in gb_models.items():
    evaluate_model(model, X_test, y_test, name)

===== RF_model_1 =====
Accuracy: 0.9024
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88        16
           1       0.92      0.92      0.92        25

    accuracy                           0.90        41
   macro avg       0.90      0.90      0.90        41
weighted avg       0.90      0.90      0.90        41

Confusion Matrix:
[[14  2]
 [ 2 23]]
----------------------------------------
===== RF_model_2 =====
Accuracy: 0.9024
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88        16
           1       0.92      0.92      0.92        25

    accuracy                           0.90        41
   macro avg       0.90      0.90      0.90        41
weighted avg       0.90      0.90      0.90        41

Confusion Matrix:
[[14  2]
 [ 2 23]]
----------------------------------------
===== RF_model_3 =====
Accuracy: 0.9024
Classification Report:
 

### 6. Итоги и сравнительная оценка

RandomForestClassifier и GradientBoostingClassifier обе являются эффективными алгоритмами для данной задачи классификации, демонстрируя высокую точность (Accuracy более 0.75) и сбалансированные показатели Precision и Recall для обоих классов.

### RandomForestClassifier

RF_model_1 показал лучшие результаты среди моделей случайного леса, благодаря оптимальному сочетанию количества деревьев, глубины и числа признаков.
Однако GradientBoostingClassifier с GB_model_3 достиг аналогичного уровня точности, что делает его предпочтительным выбором при необходимости более тонкой настройки модели.

### GradientBoostingClassifier

GB_model_1 и GB_model_3 продемонстрировали отличные результаты, с GB_model_3 немного превосходящей RF_model_1 по Accuracy и Recall для класса 1.
Настройка гиперпараметров играет ключевую роль в производительности моделей. Более глубокие деревья и увеличение числа деревьев в ансамбле способствовали улучшению метрик модели.

